#### Importing the Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#### Importing the Dataset

In [4]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## we need to input a different encoding than usual because some of the movie title contains special characters that
## cannot be treated properly with the classic encoding UTF-8. 

In [5]:
movies.head()

## Column 1 = Movie ID.
## Column 2 = Movie Name.
## Column 3 = Movie Genres.

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [7]:
users.head()

## Column 1 = User ID.
## Column 2 = Gender.
## Column 3 = Age.
## Column 4 = Some code's that corresponds to the user's job.
## Column 5 = Zip Code.

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [9]:
ratings.head()

## Column 1 = User ID.
## Column 2 = Movie's ID.
## Column 3 = User's Ratings.
## Column 4 = Timestamp when each user rated the movie which is not important.

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### Preparing the Training and Test Set

In [11]:
## U1 base and U1 test is one train-test-split of the whole dataset composed of the 100000 ratings and the other pairs are the other train-test-split

## What's the use to have several train-test-split ?
## Well that's to perform K-Fold Cross Validation manually afterwards to be able to train our model on the several training sets and test it on several test sets., sep = ''

training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')

In [12]:
training_set.head()

## Column 1 = User ID.
## Column 2 = Movie ID.
## Column 3 = Ratings.
## Column 4 = Timestamp which is irrelevant.

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [13]:
## Converting the training set from a datafram to array
training_set = np.array(training_set, dtype = 'int')

In [14]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [15]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

In [16]:
test_set.head()

## Column 1 = User ID.
## Column 2 = Movie ID.
## Column 3 = Ratings.
## Column 4 = Timestamp which is irrelevant.

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [17]:
test_set = np.array(test_set, dtype = 'int')

In [18]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

#### Getting the number of users and movies

In [20]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [21]:
print("Total Users in the U1 dataset:",nb_users)
print("Total Movies in the U1 dataset:",nb_movies)

Total Users in the U1 dataset: 943
Total Movies in the U1 dataset: 1682


#### Converting the data into an array with users in line and movies in column

In [23]:
## Since we are going to use Torch afterwards we will create a list of lists instead of a 2D numpy array.
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        user_data = data[data[:, 0] == id_users]
        id_movies = user_data[:, 1]  
        id_ratings = user_data[:, 2]
        ratings = np.zeros(nb_movies)
        ratings[id_movies.astype(int) - 1] = id_ratings
        new_data.append(ratings)
    return new_data

In [24]:
training_set = convert(training_set)

In [25]:
training_set

[array([0., 3., 4., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 2., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 

In [26]:
test_set = convert(test_set)

In [27]:
test_set

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 3., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 3., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

#### Converting the data into Torch Tensors

In [29]:
## torch.Tensor is a Multi-Dimensional Matrix with a single type. And since we are taking the FloatTensor class the single type is going to be float.
training_set = torch.FloatTensor(training_set)

/var/folders/bv/wjtkl6xj4pvd7khmgf6sd97c0000gn/T/ipykernel_20561/642618655.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  training_set = torch.FloatTensor(training_set)


In [30]:
test_set = torch.FloatTensor(test_set)

#### Creating the architecture of the Neural Network

In [32]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [33]:
sae = SAE()

In [34]:
criterion = nn.MSELoss()

In [35]:
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)
## weight_decay is used to the learning rate after every few epochs, in order to regulate the convergence.

#### Training the SAE

In [37]:
nb_epoch = 300
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0. ## this keeps track of the users who rated atleast 1 movie
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            ## mean_corrector represents the average of the error, but by only comsidering the movies which were rated.
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
            ## backward decides the direction to which the weights wil be updated i.e., will they be increased or decreased.
            ## optimizer decides the intensity of the updates i.e., the amounts by which the weights will be updated.
    print('Epoch: '+str(epoch)+' Loss: '+str(train_loss/s))

Epoch: 1 Loss: 1.7696808215035849
Epoch: 2 Loss: 1.0966075545810314
Epoch: 3 Loss: 1.0534147521467543
Epoch: 4 Loss: 1.0383601042929558
Epoch: 5 Loss: 1.0307901350382054
Epoch: 6 Loss: 1.0266676380491475
Epoch: 7 Loss: 1.0237436527657637
Epoch: 8 Loss: 1.022003961374604
Epoch: 9 Loss: 1.0205057586884971
Epoch: 10 Loss: 1.0198414771860345
Epoch: 11 Loss: 1.018974095356066
Epoch: 12 Loss: 1.0184066580161628
Epoch: 13 Loss: 1.0179991163377065
Epoch: 14 Loss: 1.0174551960517306
Epoch: 15 Loss: 1.0172286418051173
Epoch: 16 Loss: 1.016940542327472
Epoch: 17 Loss: 1.0167856827868027
Epoch: 18 Loss: 1.0167017785966401
Epoch: 19 Loss: 1.016196583580709
Epoch: 20 Loss: 1.016099570565533
Epoch: 21 Loss: 1.016127052887552
Epoch: 22 Loss: 1.015832586480109
Epoch: 23 Loss: 1.016075573045725
Epoch: 24 Loss: 1.0157645066308836
Epoch: 25 Loss: 1.015662028179628
Epoch: 26 Loss: 1.0157371587852737
Epoch: 27 Loss: 1.0154230796002137
Epoch: 28 Loss: 1.0149622739365096
Epoch: 29 Loss: 1.0130980880277238
Epo

#### Testing the SAE

In [39]:
test_loss = 0
s = 0. ## this keeps track of the users who rated atleast 1 movie
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae.forward(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        ## mean_corrector represents the average of the error, but by only comsidering the movies which were rated.
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
        ## backward decides the direction to which the weights wil be updated i.e., will they be increased or decreased.
        ## optimizer decides the intensity of the updates i.e., the amounts by which the weights will be updated.
print('Loss: '+str(test_loss/s))

Loss: 0.9405521634444237
